In [70]:
import pandas as pd
from sklearn.svm import LinearSVC
import numpy as np
import matplotlib.pyplot as plt
from sklearn.naive_bayes import GaussianNB
import seaborn as sns
from sklearn import metrics
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score, classification_report, roc_auc_score, confusion_matrix

In [46]:
df_train = pd.read_csv('earthquake_train.csv')
df_train.drop(['Unnamed: 0','building_id', 'district_id', 'vdcmun_id', 'ward_id'], axis = 1 , inplace = True)

In [47]:
df_test = pd.read_csv('earthquake_test.csv')
df_test.drop(['Unnamed: 0','building_id', 'district_id', 'vdcmun_id', 'ward_id'], axis = 1 , inplace = True)

Applying linear discriminant analysis

In [48]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
X_train = df_train.drop('damage_grade', axis = 1)
y_train = df_train['damage_grade']
X_test = df_test.drop('damage_grade', axis = 1)
y_test = df_test['damage_grade']
lda = LinearDiscriminantAnalysis()
lda.fit(X_train,y_train)
y_pred = lda.predict(X_test)
y_pred



array([3, 3, 3, ..., 3, 3, 3], dtype=int64)

In [50]:
y_pred1 = pd.DataFrame(y_pred)
y_pred1[0].value_counts()

3    71619
1     6026
2      429
Name: 0, dtype: int64

In [51]:
df_test['damage_grade'].value_counts()

3    65168
2     8555
1     4351
Name: damage_grade, dtype: int64

In [53]:
f1_score(y_pred,y_test, average='micro')

0.8580833568153291

In [57]:
recall_score(y_pred, y_test, average='micro')

0.8580833568153291

In [59]:
precision_score(y_pred, y_test, average='micro')

0.8580833568153291

In [61]:
cm = confusion_matrix(y_test,y_pred)

In [62]:
cm

array([[ 2942,    77,  1332],
       [ 2016,   152,  6387],
       [ 1068,   200, 63900]], dtype=int64)

In [63]:
from sklearn.ensemble import RandomForestClassifier
m1 = RandomForestClassifier(random_state=1)
m1.fit(X_train,y_train)


C:\Users\nikhil\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


<bound method ClassifierMixin.score of RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
            oob_score=False, random_state=1, verbose=0, warm_start=False)>

In [64]:
m1.score(X_test,y_test)

0.8105899531213977

In [68]:
 gnb=GaussianNB()
pred=gnb.fit(X_train,y_train).predict(X_test)
accuracy_score(y_test,pred,normalize=True)

0.8015600584061275

In [72]:
svc_model=LinearSVC(random_state=0)
pred=svc_model.fit(X_train,y_train).predict(X_test)
accuracy_score(y_test,pred,normalize=True)

C:\Users\nikhil\Anaconda3\lib\site-packages\sklearn\svm\base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


0.8591080257191895

In [80]:
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
KNN=KNeighborsClassifier(n_neighbors=1, weights='uniform')
NB=GaussianNB()
LR_bag=BaggingClassifier(base_estimator=lda,n_estimators=25,random_state=0,n_jobs=-1)
NB_bag=BaggingClassifier(base_estimator=NB,n_estimators=24,random_state=0,n_jobs=-1)
KNN_bag=BaggingClassifier(base_estimator=KNN,n_estimators=80,random_state=0,n_jobs=-1)
DT_reg=DecisionTreeClassifier(max_depth=10,criterion='gini',min_samples_leaf=10,min_samples_split=60)
DT_bag=BaggingClassifier(n_estimators=10,random_state=0,n_jobs=-1)
RF=RandomForestClassifier(n_estimators=130,criterion='entropy',random_state=0,n_jobs=-1)

In [84]:
from sklearn.model_selection import KFold
from sklearn.metrics import roc_curve, auc
kf=KFold(n_splits=5,shuffle=True,random_state=0)
for model, name in zip([LR_bag,NB_bag,KNN_bag,DT_reg,DT_bag], 
      ['BaggedLR','BaggedNB','BaggedKNN','DT_Reg','BaggedDT']):
    roc_auc=[]
    for train,test in kf.split(X_train,y_train):
       
        model.fit(X_train,y_train)
        y_predict=model.predict(X_test)
        #cm=metrics.confusion_matrix(Ytest,Y_predict)
        fpr,tpr, _ = roc_curve(y_test,y_predict)
        roc_auc.append(auc(fpr, tpr))
    print("AUC scores: %0.02f (+/- %0.5f) [%s]" % (np.mean(roc_auc),
                                    np.var(roc_auc,ddof=1), name )) 

MemoryError: 